In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
from helper import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']

In [2]:
def discriminator(data_dim=186, beat_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    D_in = Input(shape=[data_dim,1])
    inp1 = Concatenate()([D_in, in_label])

    x = Conv1D(filters=48, kernel_size=19, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=64, kernel_size=15, padding='same', strides=3, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=80, kernel_size=11, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=96, kernel_size=9, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=112, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = GlobalAveragePooling1D()(x)

    x1 = Conv1D(filters=48, kernel_size=9, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=64, kernel_size=7, padding='same', strides=3, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=80, kernel_size=5, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=96, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=112, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = GlobalAveragePooling1D()(x1)

    xx = concatenate([x,x1])

    xx = Dense(100)(xx)
    xx = Dense(100)(xx)

    out = Dense(1, activation='sigmoid')(xx)

    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model
    # model.summary()

# d_model = discriminator(data_dim=186, beat_dim=186)
# plot_model(d_model, to_file='temp.png', show_shapes=True)

In [3]:
def generator(noise_dim=186, beat_dim=186, out_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    G_in = Input(shape=[noise_dim,1])
    x = Concatenate()([G_in, in_label])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=2, strides=2, padding='valid', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('tanh')(x)
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=186, beat_dim=186, out_dim=186)
# plot_model(g_model, to_file='temp.png', show_shapes=True)

In [4]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='Final/gan.pdf', show_shapes=True)

In [5]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [6]:
def load_real_samples():
    X = np.load('Data/ForGAN/X.npy')
    y = np.load('Data/ForGAN/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [7]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = keras.utils.to_categorical(np.hstack((y_N[i_N], y_S[i_S], y_V[i_V])))
    # print (labels.shape)
    
    # generate class labels
    y = reshape(np.random.uniform(0.8, 1, n_samples))

    return [X, labels], y

In [8]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    np.random.shuffle(labels_fake)
    return [reshape(X_fake), keras.utils.to_categorical(labels_fake)]

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    ecgs = generator.predict([z_input, z_input])
    # create class labels
    y = reshape(np.random.uniform(0, 0.2, n_samples))

    return [ecgs, labels_input], y

In [10]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [11]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [12]:
def save_new_plot(X_R, z_input, n_batch, name):
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    # plt.show()
    plt.savefig(name, dpi=75)
    plt.close()

In [13]:
# size of the latent space
latent_dim = 186
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=data, beat_dim=data, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

folder_name = 'Final/Beat_Input/'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

plot_model(d_model, to_file=folder_name+'disc.pdf', show_shapes=True)
plot_model(g_model, to_file=folder_name+'gen.pdf', show_shapes=True)
plot_model(gan_model, to_file=folder_name+'gan.pdf', show_shapes=True)


# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)

plt.ioff()
    
filename = folder_name + 'Plots'
if not os.path.isdir(filename):
    os.mkdir(filename)

model_name = folder_name + 'Model/'
if not os.path.isdir(model_name):
    os.mkdir(model_name)

f = open(folder_name + 'Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

f = open(folder_name + 'Stats.csv', 'w')
for i in range(3):
    for mtc in metric_to_calculate:
        f.write(str(mtc)+'_'+str(i)+',')
f.write('\n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, X_real], y_real)
        
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, X_fake], y_fake)
        
        # prepare points in latent space as input for the generator
        [z_input, _] = generate_latent_points(latent_dim, n_batch)
        [X, _], _ = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_batch)
        # print (z_input.shape)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.8, 1, n_batch))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, X], y_gan)
        
#         if g_loss < G_L:
#             G_L = g_loss
#             g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')

        f = open(folder_name + 'Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()

        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i, j, bat_per_epo, d_loss1, d_loss2, g_loss))
            name = filename+'/'+str(i*1000 + j)+'.jpg'
            # generate ECGs
            z_input  = g_model.predict([z_input, X])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            save_new_plot(X_R, z_input, n_batch, name)
            g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')
            
        if (j+1)%10 == 0:
            evaluate(X, z_input, classes, metric_to_calculate, n_batch, folder_name, samples=4)

>0, 1/80, d1=0.43123, d2=1.23404 g=0.41893
>0, 3/80, d1=0.69851, d2=0.59896 g=1.11571
>0, 5/80, d1=0.94723, d2=0.49260 g=0.99789
>0, 7/80, d1=0.79130, d2=0.54421 g=0.90162
>0, 9/80, d1=0.73900, d2=0.48653 g=0.82177
>0, 11/80, d1=0.67116, d2=0.44193 g=0.68087
>0, 13/80, d1=0.59109, d2=0.51439 g=0.68262
>0, 15/80, d1=0.52059, d2=0.56807 g=0.63171
>0, 17/80, d1=0.77645, d2=0.46083 g=1.08061
>0, 19/80, d1=0.64086, d2=0.35679 g=0.62802
>0, 21/80, d1=0.48687, d2=0.42539 g=0.59504
>0, 23/80, d1=0.50331, d2=0.33484 g=0.48040
>0, 25/80, d1=0.42029, d2=0.44335 g=0.49474
>0, 27/80, d1=0.43937, d2=0.42185 g=0.42326
>0, 29/80, d1=0.42809, d2=0.32842 g=0.36781
>0, 31/80, d1=0.36859, d2=0.35872 g=0.36772
>0, 33/80, d1=0.35481, d2=0.36613 g=0.35331
>0, 35/80, d1=0.36385, d2=0.35404 g=0.35887
>0, 37/80, d1=0.37847, d2=0.34437 g=0.34323
>0, 39/80, d1=0.37146, d2=0.31501 g=0.33937
>0, 41/80, d1=0.36941, d2=0.32617 g=0.34835
>0, 43/80, d1=0.34892, d2=0.33522 g=0.33943
>0, 45/80, d1=0.34539, d2=0.34801 g=0

>4, 55/80, d1=0.30647, d2=0.33085 g=0.32188
>4, 57/80, d1=0.33509, d2=0.33153 g=0.32269
>4, 59/80, d1=0.31878, d2=0.33345 g=0.32600
>4, 61/80, d1=0.32501, d2=0.33142 g=0.32720
>4, 63/80, d1=0.32315, d2=0.32092 g=0.32733
>4, 65/80, d1=0.32472, d2=0.31568 g=0.33971
>4, 67/80, d1=0.32719, d2=0.32049 g=0.32436
>4, 69/80, d1=0.33005, d2=0.31893 g=0.32689
>4, 71/80, d1=0.31952, d2=0.33341 g=0.32703
>4, 73/80, d1=0.32502, d2=0.30740 g=0.32948
>4, 75/80, d1=0.32469, d2=0.33086 g=0.33783
>4, 77/80, d1=0.32903, d2=0.33522 g=0.32573
>4, 79/80, d1=0.33981, d2=0.31432 g=0.32237
>5, 1/80, d1=0.32828, d2=0.32929 g=0.32673
>5, 3/80, d1=0.30502, d2=0.32308 g=0.32168
>5, 5/80, d1=0.32429, d2=0.32926 g=0.33990
>5, 7/80, d1=0.32241, d2=0.32445 g=0.33178
>5, 9/80, d1=0.33795, d2=0.31703 g=0.32579
>5, 11/80, d1=0.33256, d2=0.33801 g=0.32528
>5, 13/80, d1=0.32250, d2=0.34113 g=0.32722
>5, 15/80, d1=0.34257, d2=0.32921 g=0.34118
>5, 17/80, d1=0.31522, d2=0.31932 g=0.31928
>5, 19/80, d1=0.30972, d2=0.33651 g=0

>9, 29/80, d1=0.31511, d2=0.32372 g=0.31995
>9, 31/80, d1=0.35853, d2=0.31663 g=0.35507
>9, 33/80, d1=0.36206, d2=0.33029 g=0.33232
>9, 35/80, d1=0.34699, d2=0.32185 g=0.32847
>9, 37/80, d1=0.31940, d2=0.33363 g=0.33228
>9, 39/80, d1=0.34833, d2=0.34070 g=0.32481
>9, 41/80, d1=0.32289, d2=0.32321 g=0.31682
>9, 43/80, d1=0.32901, d2=0.33077 g=0.32898
>9, 45/80, d1=0.34533, d2=0.31973 g=0.33878
>9, 47/80, d1=0.30873, d2=0.32587 g=0.32267
>9, 49/80, d1=0.32499, d2=0.33153 g=0.32351
>9, 51/80, d1=0.32165, d2=0.32721 g=0.34015
>9, 53/80, d1=0.33457, d2=0.33234 g=0.33132
>9, 55/80, d1=0.33954, d2=0.31173 g=0.33257
>9, 57/80, d1=0.33376, d2=0.34945 g=0.32131
>9, 59/80, d1=0.36489, d2=0.31595 g=0.34515
>9, 61/80, d1=0.32577, d2=0.33321 g=0.33502
>9, 63/80, d1=0.33122, d2=0.31861 g=0.31926
>9, 65/80, d1=0.30932, d2=0.33048 g=0.31643
>9, 67/80, d1=0.33406, d2=0.34401 g=0.31859
>9, 69/80, d1=0.33880, d2=0.33832 g=0.33300
>9, 71/80, d1=0.31941, d2=0.32629 g=0.32373
>9, 73/80, d1=0.31495, d2=0.3342